In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time

# List of URLs and their corresponding category names
categories = [
    {
        'url': "https://www.flipkart.com/automotive/car-bike-accessories/car-electronics-accessories/car-mobile-accessories/pr?sid=0hx,bwd,tko,av1&otracker=categorytree&otracker=nmenu_sub_Sports%2C%20Books%20%26%20More_0_Car%20Mobile%20Accessories",
        'category': 'Car Mobile Accessories'
    },
    {
        'url': "https://www.flipkart.com/beauty-and-grooming/body-face-skin-care/body-and-face-care/pr?sid=g9b%2Cema%2C5la&p%5B%5D=facets.ideal_for%255B%255D%3DMen%2B%2526%2BWomen&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DWomen&p%5B%5D=facets.ideal_for%255B%255D%3DUnisex&p%5B%5D=facets.serviceability%5B%5D%3Dtrue&otracker=nmenu_sub_Women_0_Skin%20Care",
        'category': 'Skin Care'
    },
    {
        'url': "https://www.flipkart.com/furniture/mattresses/pr?sid=wwe,rg9&otracker=nmenu_sub_Home%20%26%20Furniture_0_Mattresses",
        'category': 'Mattresses'
    },
    {
        'url': "https://www.flipkart.com/sports/badminton/pr?sid=abc,egs&p[]=facets.serviceability%5B%5D%3Dtrue&otracker=categorytree&otracker=nmenu_sub_Sports%2C%20Books%20%26%20More_0_Badminton",
        'category': 'Badminton'
    },
    {
        'url': "https://www.flipkart.com/baby-care/diaper-potty-training/baby-diapers/pr?sid=kyh,fdp,yvf&p[]=facets.serviceability%5B%5D%3Dtrue&otracker=categorytree&otracker=nmenu_sub_Baby%20%26%20Kids_0_Diapers",
        'category': 'Baby Diapers'
    }
]

# Setting up Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# This is the function to scrape data from a given URL
def scrape_flipkart(url, category):
    driver.get(url)
    time.sleep(3)  # Wait for the page to load to avoid the error.

    products = []
    count = 0

    # Try to load more products by scrolling down the page to avoid the error.
    for _ in range(2):  # Scroll a few times to load more products
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
    try:
        titles = driver.find_elements(By.CSS_SELECTOR, '.wjcEIp')
        ratings = driver.find_elements(By.CSS_SELECTOR, '.XQDdHH')
        #reviews = driver.find_elements(By.CSS_SELECTOR, '.col.EPCmJX, RcXBOT')
        reviews = driver.find_elements(By.CSS_SELECTOR, '.col.EPCmJX, RcXBOT')

        # Scrape first 20 items
        for i in range(min(20, len(titles))):
            product = {
                'Category': category,
                'Item': titles[i].get_attribute('title') if i < len(titles) else 'N/A',
                'Rating': ratings[i].text.strip() if i < len(ratings) else 'N/A',
                'Reviews': reviews[i].text.strip() if i < len(reviews) else 'N/A'
            }
            products.append(product)
            count += 1
            if count >= 20:
                break

    except Exception as e:
        print(f"Error scraping: {e}")

    return products

# saving data to a CSV file
def write_to_csv(data, filename='scraped_products.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['Category', 'Item', 'Rating', 'Reviews'])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# Scrape each URL and collect data
# Initialize an empty list to store all the product data
all_product_data = []

for category_info in categories:
    # Firstly, get the URL and category name from the category_info dictionary
    url = category_info['url']
    category = category_info['category']
    
    # Print the scraping status.
    print(f"Scraping data from: {category} - {url}")
    
    # Call the scrape_flipkart function to get product data from the URL for this category
    data = scrape_flipkart(url, category)
    
    # Add the product data to the all_product_data list
    all_product_data.extend(data)
    
    # Print the product details: Category, Title, Rating, and Reviews
    for product in data:
        print(f"Category: {product['Category']}, Title: {product['Item']}, Rating: {product['Rating']}, Reviews: {product['Reviews']}")
    
    # Print a separator line after each category's data is scraped
    print("\n" + "-"*50 + "\n")

# Write the final data to CSV
write_to_csv(all_product_data)

# Close the WebDriver
driver.quit()


Scraping data from: Car Mobile Accessories - https://www.flipkart.com/automotive/car-bike-accessories/car-electronics-accessories/car-mobile-accessories/pr?sid=0hx,bwd,tko,av1&otracker=categorytree&otracker=nmenu_sub_Sports%2C%20Books%20%26%20More_0_Car%20Mobile%20Accessories
Category: Car Mobile Accessories, Title: Portronics 3 Amp Turbo Car Charger, Rating: 4.4, Reviews: 5
Excellent
Plastic quality is good,build and finishing is awesom,I m happy
Mukesh kumar
Certified Buyer
Oct, 2022
Category: Car Mobile Accessories, Title: Portronics Spark Phone Charger for Scooters & Motorcycles, 33W Type C, 22.5W USB 3 A Bike Mobile Charger, Rating: 4.2, Reviews: 5
Highly recommended
Product is good bt build quality is ok ok
Flipkart Customer
Certified Buyer
1 day ago
Category: Car Mobile Accessories, Title: Otoroys Universal Bike Mount Holder Verson 2.0 Metal Body 360 Degree Bike Mobile Holder, Rating: 3.8, Reviews: 4
Very Good
Good quality and best price
Shivam Kumar
Certified Buyer
Apr, 2024
Ca